In [106]:
import pandas as pd
import requests
import json
import time
import pickle

from tqdm.notebook import tqdm

In [2]:
!ls data

ivs2021_attendees.csv


In [4]:
channel_ids = {
    "room-a": "C01QZKE2Y2J",
    "room-b": "C01RCHRLJPN",
    "room-c": "C01QZKECPK8",
    "room-d": "C01RCHRRP8C",
    "ovice-visitor": "C01R3DPL8LD",
    "chat-en": "C01R6CU031Q",
    "chat": "C01R6JUMP7D",
    "support": "C01R3JFKGLW",
}

url = "http://localhost:8080/onboard"
GOOGLE_APP_TOKEN = "izSJx657D16RF0tD5$x"

In [147]:
ivs = pd.read_csv("data/ivs2021_attendees.csv")
# if email duplicate, retain the newest
ivs = ivs.drop_duplicates('メールアドレス', keep='last')
ivs = ivs.fillna('')

In [148]:
out_dict_list = []
for row in ivs.iterrows():
    in_dict = row[1].to_dict()
    out_dict = {"username": "{} {}".format(in_dict['名(ローマ字)'], in_dict['姓(ローマ字)']),
                "username_jp": "{} {}".format(in_dict['姓'], in_dict['名']), 
                "email": in_dict['メールアドレス'], 
                "company_name": in_dict['勤務先名/学校名(英語表記)'].replace("\'", ""),
                "title": in_dict['役職'], 
                "category": "", 
                "industry": in_dict['業種'],
                "fb_url": "",
                "channels": "{{{}}}".format(", ".join(channel_ids.values())),  # currently add to all channels
                "profile_img": None,
                "userrole": ""}
    out_dict_list.append(out_dict)

In [155]:
print(len(out_dict_list))

580


In [177]:
# get email list from pickle

with open('emails_uploaded.pickle', 'rb') as handle:
    emails_uploaded = pickle.load(handle)

for data in tqdm(out_dict_list, total=len(out_dict_list)):
    data = {k: v for k, v in data.items() if v}
    if data['email'] in emails_uploaded:
        continue
    # check if email already uploaded, if yes, skip.
    r = requests.get(url, json={"event": {"type": "google_sheet_input", "data": data}, "token": GOOGLE_APP_TOKEN})
    emails_uploaded.append(r.json()['email'])
    time.sleep(0.25)

    
# save email list to pickle right away
with open('emails_uploaded.pickle', 'wb') as handle:
    emails_uploaded = list(set(emails_uploaded))
    pickle.dump(emails_uploaded, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/580 [00:00<?, ?it/s]

In [176]:
# save email list to pickle right away
with open('emails_uploaded.pickle', 'wb') as handle:
    emails_uploaded = list(set(emails_uploaded))
    pickle.dump(emails_uploaded, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Debug Zone

In [36]:
ian_data = {
    "username": "Ian Su",
    "username_jp": "蘇 倚恩",
    "email": "ian@infinityventures.com",
    "company_name": "Infinity Ventures",
    "title": "Tech Lead",
    "category": "VC",
    "industry": "IT",
    "channels": "{C01RCPTPU9H}",
    "fb_url": "https://www.facebook.com/ensu.tw",
    "userrole": "Staff",
    "profile_check": False,
    "photo_check": False
}

tiff_data = {
    "username": "Tiffany Liu",
    "username_jp": "ティファニー・リュウ",
    "email": "tiffany@infinityventures.com",
    "company_name": "Infinity Ventures",
    "title": "Japan Community Manager",
    "category": "VC",
    "industry": "IT",
    "channels": "{C01RCPTPU9H}",
    "fb_url": "",
    "userrole": "Staff",
    "profile_check": False,
    "photo_check": False   
}

In [20]:
r = requests.get(url, json={"event": {"type": "google_sheet_input", "data": tiff_data}, "token": GOOGLE_APP_TOKEN})
